# Meetup index generator

This notebook generates an Elasticsearch index with information about the *rvsps* for a set of Meetup groups defined in the `settings.yml` file. You'll need a [Meetup API token](https://secure.meetup.com/meetup_api/key/)

In [ ]:
import utils

#utils.logging.basicConfig(level=utils.logging.INFO)

In [ ]:
from perceval.backends.core.meetup import Meetup

In [ ]:
#Reading settings file and establishing connection with an Elasticsearch

settings = utils.read_config_file('settings.yml')
es = utils.establish_connection(settings['es_host'])

In [ ]:
# ES mapping for a basic Meetup index

MAPPING_MEETUP = {
    "mappings": {
        "item": {
            "properties": {
                "date": {"type": "date"},
                "member_id": {"type": "keyword"},
                "member_name": {"type": "keyword"},
                "rsvp_response": {"type": "keyword"},
                "meetup_name": {"type": "keyword"},
                "meetup_id": {"type": "keyword"},
                "meetup_venue": {"type": "keyword"},
                "group": {"type": "keyword"}
            }
        }
    }
}

**WARNING**: We remove/delete any existing `meetup` index. So if you wanna keep an existing one, just change the name for the new one.

In [ ]:
# Creaiting meetup index in the Elasticsearch using defined mapping

utils.create_ES_index(es, 'meetup', MAPPING_MEETUP)

In [ ]:
# For each Meetup groud defined in the settings file...

for group_name in settings['meetup']:
    group = Meetup(group=group_name, sleep_for_rate=True, api_token=settings['meetup_api_key'])
    
    for item in group.fetch():
        """" Getting items from Meetup using Perceval
        
        For Meetup, Perceval items are Meetup meetings
        """"
        
        for rsvp in item['data']['rsvps']:
            """" For each RSVP in the Meetup meeting
            
            Get summary information
            """"
            
            summary = {
                'date': rsvp['created'],
                'member_id': rsvp['member']['id'],
                'member_name': rsvp['member']['name'],
                'rvsp_response': rsvp['response'],
                'meetup_name': item['data']['name'],
                'meetup_id': item['data']['id'],
                'group': item['data']['group']['name']
            }
            if 'venue' in item['data'].keys():
                summary['meetup_venue'] = item['data']['venue']['name']
            
            # Store RSVP information in the Elasticsearch index created
            
            es.index(index='meetups', doc_type='item', body=summary)